In [30]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
atc = pd.read_csv("./data/의약품등제품정보목록_ATC_v2.csv")

In [3]:
pd.set_option("display.max_columns", 100)

In [5]:
atc

,품목기준코드,제품명,제품영문명,업체명,업체영문명,허가일,품목구분,허가번호,취소/취하,취소/취하일자,주성분,첨가제,품목분류,전문의약품,완제/원료,허가/신고,제조/수입,마약구분,모양,색상,제형,장축,단축,신약구분,표준코드명,보험코드,ATC코드,타겟,약효,약물학적특성,개별성분,묶음의약품정보,e은약요,수입제조국,주성분영문,보험약가
0,200511943,1.셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물)2.셀레나제티퍼오랄액(아셀...,selenase 100 microgram peroral(sodium selenite...,비오신코리아(주),biosynkorea,2005-12-27,의약품,1,정상,NaN,아셀렌산나트륨오수화물,"염화나트륨,희석염산,정제수",NaN,일반의약품,완제의약품,신고,수입,NaN,NaN,NaN,NaN,NaN,NaN,N,"8806748000102,8806748000119,8806748000126,8806...",674800010,A12CE02,A,12,C,2.0,NaN,200511943.0,독일,Sodium Selenite Pentahydrate,NaN
1,201701571,1.셀레뉴원오랄액(아셀렌산나트륨오수화물) 2.셀큐민185오랄액(아셀렌산나트륨오수화물),Selenew-one Oral solution,(주)한국비엠아이,BMI KOREA,2017-03-31,의약품,5047,정상,NaN,아셀렌산나트륨오수화물,"심황색소,인산이수소나트륨이수화물,인산수소나트륨수화물,염화나트륨,정제수,D-소르비톨",NaN,일반의약품,완제의약품,신고,제조,NaN,NaN,NaN,NaN,NaN,NaN,N,"8806548022908,8806548022922,8806548022915,8806...",654802290,A12CE02,A,12,C,2.0,NaN,201701571.0,NaN,Sodium Selenite Pentahydrate,NaN
2,199200813,1.영진구론산바몬드액2.영진구론산바몬드스파클링액,1. Yungjin Guronsan Varmond Sol. 2.Yungjin Gur...,해태에이치티비(주),"HAITAI htb Co., Ltd.",1992-06-27,의약품,1,정상,NaN,글루쿠로노락톤/오로트산수화물/카페인무수물/니코틴산아미드/리보플라빈/피리독신염산염,"시트르산수화물,이성화당,염화나트륨,백당,에탄올(95),파인애플엣센스에스,시트르산나트...",NaN,일반의약품,완제의약품,신고,제조,NaN,NaN,NaN,NaN,NaN,NaN,N,"8806275000200,8806424006909,8806424006916,8806...",627500020,A13A,A,13,A,NaN,NaN,199200813.0,NaN,Glucuronolactone/Orotic Acid Hydrate/Anhydrous...,NaN
3,201500388,"D3베이스주 100,000 IU (콜레칼시페롤)","D3BASE 100,000I.U./1ml solution for injection",(주)메디포트,"MEDIPORT Co.,Ltd.",2015-01-26,의약품,1,정상,NaN,콜레칼시페롤,정제올리브유,NaN,전문의약품,완제의약품,허가,수입,NaN,NaN,NaN,NaN,NaN,NaN,N,"8800500000102,8800500000126,8800500000119",50000010,A11CC05,A,11,C,5.0,NaN,NaN,이탈리아,Cholecalciferol,NaN
4,201500389,"D3베이스주 300,000 IU (콜레칼시페롤)","D3BASE 300,000I.U./1ml solution for injection",(주)메디포트,"MEDIPORT Co.,Ltd.",2015-01-26,의약품,2,정상,NaN,콜레칼시페롤,정제올리브유,NaN,전문의약품,완제의약품,허가,수입,NaN,NaN,NaN,NaN,NaN,NaN,N,"8800500000201,8800500000225,8800500000218",50000020,A11CC05,A,11,C,5.0,NaN,NaN,이탈리아,Cholecalciferol,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7145,202106959,힐러스메트정,Healusmet Tabs. 50/500 mg,한국유나이티드제약(주),Korea United Pharm.,2021-10-18,의약품,1246,정상,NaN,빌다글립틴/메트포르민염산염,"오파드라이 AMB II 88A620064 노란색,스테아르산마그네슘,히드록시프로필셀룰로오스",NaN,전문의약품,완제의약품,허가,제조,NaN,타원형,NaN,NaN,17.70,7.40,N,"8806443096608,8806443096615",644309660,A10BD08,A,10,B,8.0,202106959.0,NaN,NaN,Vildagliptin/Metformin Hydrochloride,NaN
7146,202200168,힐러스정(빌다글립틴),Healus Tab. 50mg,한국유나이티드제약(주),Korea United Pharm.,2022-01-10,의약품,1247,정상,NaN,빌다글립틴,"미결정셀룰로오스,유당수화물,스테아르산마그네슘,전분글리콜산나트륨",NaN,전문의약품,완제의약품,허가,제조,NaN,원형,NaN,NaN,6.50,6.50,N,"8806443096707,8806443096714",644309670,A10BH02,A,10,B,2.0,202200168.0,NaN,NaN,Vildagliptin,NaN
7147,201503211,힐콘정(폴리카르보필칼슘),Hylcon Tab.,(주)한국파마,"KOREA PHARMA Co., Ltd.",2015-05-27,의약품,5090,정상,NaN,폴리카르보필칼슘,"크로스포비돈,히프로멜로오스2910,피마자유,카라멜색소,스테아르산마그네슘,경질무수규산...",NaN,전문의약품,완제의약품,허가,제조,NaN,타원형,NaN,NaN,19.00,9.00,N,"8806530056607,8806530056614,8806530056621",653005660,A06AC08,A,6,A,8.0,NaN,NaN,NaN,Calcium Polycarbophil,NaN
7148,201403757,힘톤플러스연질캡슐,Himton Plus Soft Cap.,(주)메디카코리아,"MEDICA KOREA Co., Ltd.",2014-08-13,의약품,5028,정상,NaN,레티놀팔미테이트/아스코르브산/토코페롤아세테이트/셀레늄함유건조효모/산화아연/산화마그네...,"비결정성 소르비톨액,콩기름,레시틴,치자청색소,에틸바닐린,숙신산젤라틴,산화티탄,카카오...",NaN,일반의약품,완제의약품,신고,제조,NaN,타원형,NaN,NaN,24.65,9.13,N,"8806460038001,8806460038018,8806460038025",646003800,A11JB,A,11,J,NaN,NaN,201403757.0,NaN,Retinol Palmitate/Ascorbic Acid/Tocopherol Ace...,NaN


In [6]:
insure = atc['보험코드']

In [7]:
insure = insure.to_list()

In [8]:
len(insure)

7150

In [26]:
atc[atc['보험코드'] == 649508430]

,품목기준코드,제품명,제품영문명,업체명,업체영문명,허가일,품목구분,허가번호,취소/취하,취소/취하일자,주성분,첨가제,품목분류,전문의약품,완제/원료,허가/신고,제조/수입,마약구분,모양,색상,제형,장축,단축,신약구분,표준코드명,보험코드,ATC코드,타겟,약효,약물학적특성,개별성분,묶음의약품정보,e은약요,수입제조국,주성분영문,보험약가
2743,202400391,본에디칼정,Bonedical Tab,유니메드제약(주),UNIMED PHARM INC.,2024-02-23,의약품,5157,정상,NaN,탄산칼슘/농축콜레칼시페롤분말,"미결정셀룰로오스,유당수화물,스테아르산마그네슘,카르복시메틸셀룰로오스칼슘,히드록시프로필...",NaN,일반의약품,완제의약품,신고,제조,NaN,장방형,NaN,NaN,12.1,6.1,N,"8806495084301,8806495084318,8806495084325",649508430,A12AX,A,12,A,NaN,NaN,NaN,NaN,Calcium Carbonate/Cholecalciferol Concentrate ...,NaN


In [25]:
insure[2743]

649508430

In [10]:
options = Options()
options.add_argument('--disable-images')
options.add_experimental_option("prefs", {'profile.managed_default_content_settings.images': 2})
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument("--window-size=1920x1080")
options.add_argument('--headless=new') 
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--log-level=3')
options.add_argument('--disable-gpu')
options.add_argument('--incognito')

In [ ]:
649508430	
https://www.hira.or.kr/ra/medi/getPermissionInfo.do?pgmid=HIRAA030029000000&medcCd=649508430&kfdaMedcCd

In [64]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

pill_name_list = []
pill_name_en_list = []
code_list = []
insure_money_list = []
po_list = []

num = 2745
for i in range(2744, len(insure)):
    url = f"https://www.hira.or.kr/ra/medi/getPermissionInfo.do?pgmid=HIRAA030029000000&medcCd={insure[i]}&kfdaMedcCd="

    driver.get(url)


    pill_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tmplcode"]/table[1]/tbody/tr[2]/td'))).text
    pill_name_en = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tmplcode"]/table[1]/tbody/tr[3]/td'))).text
    code = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tmplcode"]/table[1]/tbody/tr[6]/td'))).text
    insure_money = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tmplcode"]/table[1]/tbody/tr[7]/td'))).text
    po = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH , '//*[@id="tmplcode"]/table[1]/tbody/tr[10]/td'))).text

    print("No :", num)
    print("약 이름 :", pill_name)
    print("약 이름(영문) :", pill_name_en)
    print("품목기준코드 :", code)
    print("보험상한가 :", insure_money)
    print("포장단위 :", po)

    pill_name_list.append(pill_name)
    pill_name_en_list.append(pill_name_en)
    code_list.append(code)
    insure_money_list.append(insure_money)
    po_list.append(po)

    num += 1
    
driver.close()

No : 2745
약 이름 : 본에존연질캡슐
약 이름(영문) : Bone-E-Jon Soft Cap.
품목기준코드 : 201000163
보험상한가 : 원
포장단위 : 60캡슐(10캡슐/PTP포장 x 6), 120캡슐((10캡슐/PTP포장 x 6) x 2)
No : 2746
약 이름 : 본에킹연질캡슐
약 이름(영문) : Boneking Soft Cap.
품목기준코드 : 201104871
보험상한가 : 원
포장단위 : 60캡슐/상자[10캡슐/PTP×6],120캡슐/상자[(10캡슐/PTP×6)×2]
No : 2747
약 이름 : 본에탑연질캡슐
약 이름(영문) : Bone-etop soft capsules
품목기준코드 : 201902441
보험상한가 : 원
포장단위 : 60캡슐/상자[(10캡슐/피티피X6)]
No : 2748
약 이름 : 본엔디연질캡슐
약 이름(영문) : Bone&D Soft Cap.
품목기준코드 : 201007524
보험상한가 : 원
포장단위 : 60캡슐/PTP, 120캡슐/PTP
No : 2749
약 이름 : 본키서방정(칼시트리올)(수출용)
약 이름(영문) : Bonky Tab. (Calcitriol)
품목기준코드 : 200200213
보험상한가 : 원
포장단위 : 30정(10정/PTPX3)
No : 2750
약 이름 : 본키씨알정(칼시트리올)(수출용)
약 이름(영문) : Bonky CR Tab. (Calcitriol)
품목기준코드 : 200000365
보험상한가 : 원
포장단위 : 30정(10정/PTPX3)
No : 2751
약 이름 : 본키에이연질캡슐0.25마이크로그램(알파칼시돌)(수출용)
약 이름(영문) : Bonky-A soft cap. 0.25mcg (alfacalcidol)
품목기준코드 : 201310155
보험상한가 : 원
포장단위 : 30캡슐(10캡슐/PTPX3)
No : 2752
약 이름 : 본키연질캡슐(칼시트리올)
약 이름(영문) : Bonky Soft Cap.(Calcitriol)
품목기준코드 : 199300288
보험상한가 :

In [65]:
a03_insure_01 = pd.DataFrame({'약명': pill_name_list, '약명(영문)': pill_name_en_list, '품목기준코드': code_list, '보험상한가': insure_money_list, '포장단위': po_list})

In [13]:
a03_insure

,약명,약명(영문),품목기준코드,보험상한가,포장단위
0,1.셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물)2.셀레나제티퍼오랄액(아셀...,selenase 100 microgram peroral(sodium selenite...,200511943,원,"셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물) 2㎖ X 20, 셀레나제티퍼..."
1,1.셀레뉴원오랄액(아셀렌산나트륨오수화물)2.셀큐민185오랄액(아셀렌산나트륨오수화물),Selenew-one Oral solution,201701571,원,"2mL/포 X 15, 2mL/포 X 20"
2,1.영진구론산바몬드액2.영진구론산바몬드스파클링액,1. Yungjin Guronsan Varmond Sol. 2.Yungjin Gur...,199200813,원,100ml/병×자사포장단위
3,D3베이스주100000IU(콜레칼시페롤),"D3BASE 100,000I.U./1ml solution for injection",201500388,원,6앰플/박스
4,D3베이스주300000IU(콜레칼시페롤),"D3BASE 300,000I.U./1ml solution for injection",201500389,원,2앰플/박스
...,...,...,...,...,...
2738,본본연질캡슐,Bonebone Soft Cap.,201310644,원,"60캡슐(10캡슐/PTP*6), 120캡슐(10캡슐/PTP*12)"
2739,본비타디주(콜레칼시페롤),Bonvita-D Inj.,202203885,원,"10앰플/상자[1.5mL/앰플],10앰플/상자[1mL/앰플]"
2740,본스칼엠정,bonscal M tab,202002068,원,120정/병[X 2]
2741,본스칼츄어블정,Bonscal Chewable Tab.,201110424,원,"120 T, 240 T"


In [60]:
append_list = {"약명" : "본에디칼정", "약명(영문)" : "Bonedical Tab", "품목기준코드" : np.NAN, "보험상한가" : "원", "포장단위" : ""}

In [62]:
a03_test = pd.concat([a03_insure, pd.DataFrame(append_list, index=[0])], axis=0, ignore_index=True)

In [63]:
a03_test

,약명,약명(영문),품목기준코드,보험상한가,포장단위
0,1.셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물)2.셀레나제티퍼오랄액(아셀...,selenase 100 microgram peroral(sodium selenite...,200511943,원,"셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물) 2㎖ X 20, 셀레나제티퍼..."
1,1.셀레뉴원오랄액(아셀렌산나트륨오수화물)2.셀큐민185오랄액(아셀렌산나트륨오수화물),Selenew-one Oral solution,201701571,원,"2mL/포 X 15, 2mL/포 X 20"
2,1.영진구론산바몬드액2.영진구론산바몬드스파클링액,1. Yungjin Guronsan Varmond Sol. 2.Yungjin Gur...,199200813,원,100ml/병×자사포장단위
3,D3베이스주100000IU(콜레칼시페롤),"D3BASE 100,000I.U./1ml solution for injection",201500388,원,6앰플/박스
4,D3베이스주300000IU(콜레칼시페롤),"D3BASE 300,000I.U./1ml solution for injection",201500389,원,2앰플/박스
...,...,...,...,...,...
2739,본비타디주(콜레칼시페롤),Bonvita-D Inj.,202203885,원,"10앰플/상자[1.5mL/앰플],10앰플/상자[1mL/앰플]"
2740,본스칼엠정,bonscal M tab,202002068,원,120정/병[X 2]
2741,본스칼츄어블정,Bonscal Chewable Tab.,201110424,원,"120 T, 240 T"
2742,본앤본연질캡슐,Bone&Bone Soft Cap.,202401312,원,"60캡슐/피티피[10캡슐/PTP*6],120캡슐/피티피[10캡슐/PTP*12]"


In [66]:
a03_insure_01

,약명,약명(영문),품목기준코드,보험상한가,포장단위
0,본에존연질캡슐,Bone-E-Jon Soft Cap.,201000163,원,"60캡슐(10캡슐/PTP포장 x 6), 120캡슐((10캡슐/PTP포장 x 6) x 2)"
1,본에킹연질캡슐,Boneking Soft Cap.,201104871,원,"60캡슐/상자[10캡슐/PTP×6],120캡슐/상자[(10캡슐/PTP×6)×2]"
2,본에탑연질캡슐,Bone-etop soft capsules,201902441,원,60캡슐/상자[(10캡슐/피티피X6)]
3,본엔디연질캡슐,Bone&D Soft Cap.,201007524,원,"60캡슐/PTP, 120캡슐/PTP"
4,본키서방정(칼시트리올)(수출용),Bonky Tab. (Calcitriol),200200213,원,30정(10정/PTPX3)
...,...,...,...,...,...
4401,힐러스메트정,Healusmet Tabs. 50/500 mg,202106959,310원,30정/상자[(10정/PTP X3)]
4402,힐러스정(빌다글립틴),Healus Tab. 50mg,202200168,240원,30정/상자[10정/PTP X 3]
4403,힐콘정(폴리카르보필칼슘),Hylcon Tab.,201503211,89원,"30정/병, 500정/병"
4404,힘톤플러스연질캡슐,Himton Plus Soft Cap.,201403757,원,100캡슐(10캡슐/PTPX10)


In [84]:
a03_insure_02

,약명,약명(영문),품목기준코드,보험상한가,포장단위
0,티로드정(티로프라미드염산염),Tirode Tab.,200001860,135원,"30정/병, 200정/병"
1,티로디정100밀리그램(티로프라미드염산염),TIROD Tab. 100mg (Tiropramide Hydrochloride),201508307,115원,"30정/병,100정/병,300정/병"
2,티로라정(티로프라미드염산염),Tirora Tab.,200302054,115원,"30정/병, 500정/병"
3,티로랙스캡슐(티로프라미드염산염),Tirolax Capsule,200300418,115원,"30캡슐/병,100캡슐/병"
4,티로마정(티로프라미드염산염),Tiroma Tab.,200103345,115원,"30정/병, 300정/병"
...,...,...,...,...,...
117,휴니즈티로프라미드염산염정,Huniz Tiropramide HCI Tablet,201503193,115원,"30정/병, 500정/병"
118,휴리토정(이토프리드염산염),Huritor Tab.,201603577,108원,"30정/병, 500정/병"
119,휴모사정(모사프리드시트르산염수화물),Humosa Tab.,201207713,원,"30정/병, 300정/병"
120,휴토즈정(이토프리드염산염),Hutoz Tablet,201603959,92원,"30정/병, 300정/병"


In [77]:
insure[296 + 190]

671803200

In [80]:
296 + 190

486

In [95]:
a03_insure_final = pd.concat([a03_insure, a03_insure_01, a03_insure_02], ignore_index=True)

In [88]:
ls = [a03_insure_01, a03_insure_02]

In [ ]:
pd.merge(a03_insure, a03_insure_01)

In [90]:
a03_insure_con = pd.concat([a03_insure_01, a03_insure_02], axis=1)

In [71]:
a03_insure_final = pd.concat([a03_insure, a03_insure_01], axis=0, ignore_index=True)

In [72]:
a03_insure_final

,약명,약명(영문),품목기준코드,보험상한가,포장단위
0,1.셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물)2.셀레나제티퍼오랄액(아셀...,selenase 100 microgram peroral(sodium selenite...,200511943,원,"셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물) 2㎖ X 20, 셀레나제티퍼..."
1,1.셀레뉴원오랄액(아셀렌산나트륨오수화물)2.셀큐민185오랄액(아셀렌산나트륨오수화물),Selenew-one Oral solution,201701571,원,"2mL/포 X 15, 2mL/포 X 20"
2,1.영진구론산바몬드액2.영진구론산바몬드스파클링액,1. Yungjin Guronsan Varmond Sol. 2.Yungjin Gur...,199200813,원,100ml/병×자사포장단위
3,D3베이스주100000IU(콜레칼시페롤),"D3BASE 100,000I.U./1ml solution for injection",201500388,원,6앰플/박스
4,D3베이스주300000IU(콜레칼시페롤),"D3BASE 300,000I.U./1ml solution for injection",201500389,원,2앰플/박스
...,...,...,...,...,...
7144,힐러스메트정,Healusmet Tabs. 50/500 mg,202106959,310원,30정/상자[(10정/PTP X3)]
7145,힐러스정(빌다글립틴),Healus Tab. 50mg,202200168,240원,30정/상자[10정/PTP X 3]
7146,힐콘정(폴리카르보필칼슘),Hylcon Tab.,201503211,89원,"30정/병, 500정/병"
7147,힘톤플러스연질캡슐,Himton Plus Soft Cap.,201403757,원,100캡슐(10캡슐/PTPX10)


In [53]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url2 = "https://www.hira.or.kr/ra/medi/getPermissionInfo.do?pgmid=HIRAA030029000000&medcCd=654802290&kfdaMedcCd="

driver.get(url2)

In [96]:
a03_insure_final['보험상한']

,약명,약명(영문),품목기준코드,보험상한가,포장단위
0,가나릴정(이토프리드염산염),GANARILLTAB.(ITOPRIDEHCl),200402915,92원,"30정/병 - 30정/병,100정/병 - 100정/병,500정/병 - 500정/병,..."
1,가나메드정(이토프리드염산염),Ganamed Tab.(Itopride HCl),200803702,원,"30정/병,300정/병,1000정/병"
2,가나젠정(이토프리드염산염),Ganazen Tab.,202000325,92원,"30정/병,500정/병"
3,가나칸정50밀리그램(이토프리드염산염),Ganaton Tablet 50mg (Itopride Hcl),199600267,103원,"100정/병, 500정/병, 1000정/병, 100정(20정/PTPx5)"
4,가나텍정(이토프리드염산염),,202000164,92원,"30정/병,500정/병"
...,...,...,...,...,...
603,휴니즈티로프라미드염산염정,Huniz Tiropramide HCI Tablet,201503193,115원,"30정/병, 500정/병"
604,휴리토정(이토프리드염산염),Huritor Tab.,201603577,108원,"30정/병, 500정/병"
605,휴모사정(모사프리드시트르산염수화물),Humosa Tab.,201207713,원,"30정/병, 300정/병"
606,휴토즈정(이토프리드염산염),Hutoz Tablet,201603959,92원,"30정/병, 300정/병"


In [73]:
a03_copy = a03_insure_final.copy()

In [74]:
a03_copy['보험상한가'] = a03_copy['보험상한가'].apply(lambda x : x.replace("원", ""))

In [103]:
import numpy as np

In [75]:
a03_copy['보험상한가'] = a03_copy['보험상한가'].apply(lambda x : np.NAN if len(x) < 1 else x)

In [76]:
a03_copy.rename(columns={'보험상한가':'보험상한가(단위/원)'}, inplace=True)

In [77]:
a03_copy

,약명,약명(영문),품목기준코드,보험상한가(단위/원),포장단위
0,1.셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물)2.셀레나제티퍼오랄액(아셀...,selenase 100 microgram peroral(sodium selenite...,200511943,NaN,"셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물) 2㎖ X 20, 셀레나제티퍼..."
1,1.셀레뉴원오랄액(아셀렌산나트륨오수화물)2.셀큐민185오랄액(아셀렌산나트륨오수화물),Selenew-one Oral solution,201701571,NaN,"2mL/포 X 15, 2mL/포 X 20"
2,1.영진구론산바몬드액2.영진구론산바몬드스파클링액,1. Yungjin Guronsan Varmond Sol. 2.Yungjin Gur...,199200813,NaN,100ml/병×자사포장단위
3,D3베이스주100000IU(콜레칼시페롤),"D3BASE 100,000I.U./1ml solution for injection",201500388,NaN,6앰플/박스
4,D3베이스주300000IU(콜레칼시페롤),"D3BASE 300,000I.U./1ml solution for injection",201500389,NaN,2앰플/박스
...,...,...,...,...,...
7144,힐러스메트정,Healusmet Tabs. 50/500 mg,202106959,310,30정/상자[(10정/PTP X3)]
7145,힐러스정(빌다글립틴),Healus Tab. 50mg,202200168,240,30정/상자[10정/PTP X 3]
7146,힐콘정(폴리카르보필칼슘),Hylcon Tab.,201503211,89,"30정/병, 500정/병"
7147,힘톤플러스연질캡슐,Himton Plus Soft Cap.,201403757,NaN,100캡슐(10캡슐/PTPX10)


In [111]:
a03_copy['포장단위'].apply(lambda x : x.split(","))

0      [30정/병 - 30정/병, 100정/병 - 100정/병, 500정/병 - 500정...
1                               [30정/병, 300정/병, 1000정/병]
2                                        [30정/병, 500정/병]
3          [100정/병,  500정/병,  1000정/병,  100정(20정/PTPx5)]
4                                        [30정/병, 500정/병]
                             ...                        
603                                     [30정/병,  500정/병]
604                                     [30정/병,  500정/병]
605                                     [30정/병,  300정/병]
606                                     [30정/병,  300정/병]
607                [[내수용] 1mL x 50앰플,  [수출용] 1mL x 10앰플]
Name: 포장단위, Length: 608, dtype: object

In [79]:
a03_copy.to_csv("./data/보험상한가_A.csv")